In [1]:
import pandas as pd
import numpy as np
import re 
from pandas_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# for Q-Q plots
import scipy.stats as stats

# from feature-engine
#from feature_engine.imputation import MeanMedianImputer
import feature_engine.imputation as imp

# to split the datasets
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [2]:
DATA_PATH = "../data/fake_job_postings.csv"

In [3]:
postings = pd.read_csv(DATA_PATH)
postings.shape

(17880, 18)

In [4]:
ProfileReport(postings)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## Location

In [5]:
# number of missing values on location
postings.location.isnull().sum()

346

Missing values at random or not?

In [6]:
missing_location = postings[postings.location.isnull()]
missing_location.head(3)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
144,145,Forward Cap.,NaN,NaN,NaN,NaN,The group has raised a fund for the purchase o...,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,1
204,205,Junior Python Developer,NaN,Line-Up,NaN,Playfair Capital is an early stage technology ...,Who we’re looking forMaker MentalityAre you fo...,Skills and experienceDegree in Computer Scienc...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
234,235,Postgraduate Certificate in Social Innovation ...,NaN,NaN,NaN,The Amani Institute is about developing whole ...,"This unique, field-based, full-time program br...",What do we look for in a program participant?I...,Sign up for:25 classmates from around the worl...,0,1,1,NaN,NaN,NaN,NaN,NaN,0


In [7]:
# häufigkeit der verschiedenen variablen
vars = list(missing_location.columns)
vars


['job_id',
 'title',
 'location',
 'department',
 'salary_range',
 'company_profile',
 'description',
 'requirements',
 'benefits',
 'telecommuting',
 'has_company_logo',
 'has_questions',
 'employment_type',
 'required_experience',
 'required_education',
 'industry',
 'function',
 'fraudulent']

We will check each variable, to know if there is a relation between location and other variables

hint: job_id and title do NOT have relevance as they are purely for identification and do NOT give us any valuable information

In [8]:
def plot_relation(all, missing, variable):
    fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]], 
                    subplot_titles=['All Data', 'Missing location'])

    fig.add_trace(go.Pie(labels=all[variable], values=all.counts), 
                row = 1, col = 1)

    fig.add_trace(go.Pie(labels=missing[variable], values=missing.counts), 
                row = 1, col = 2)

    fig.update_layout(title_text=f'Relation between missing location and {variable}')
    fig.show()

In [9]:
def make_dataframe_based_on_counts_and_sum_low(df, variable, threshold, is_missing=False):
    if not is_missing:
        df = df[variable].value_counts().rename_axis(variable).reset_index(name='counts')
    # get the values below the threshold
    low_df = df.where(df['counts'] <= threshold).dropna().sort_values(by='counts', ascending=False)
    # count and sum
    new_row = pd.Series({variable: f'{low_df.counts.count()} other {variable}s below {threshold}', 
                        'counts': low_df.counts.sum()})
    # add the new row after removing the low values
    df = df.where(df['counts'] > threshold).dropna().sort_values(by='counts', ascending=False)
    df = pd.concat([df, 
                    pd.DataFrame([new_row], 
                                 columns=new_row.index)]
                   ).reset_index(drop=True)
    return df

In [10]:
def make_df_missing_location(df, variable):
    return df[variable].value_counts().rename_axis(variable).reset_index(name='counts')

In [11]:
missing_at_random = []
not_missing_at_random = []

#### Department

In [12]:
departments_missing_in_location = make_df_missing_location(missing_location, 'department')
departments_missing_in_location.head(3)

,department,counts
0,Editorial,5
1,Marketing,5
2,Provisions,5


In [13]:
departments_missing_in_location = make_dataframe_based_on_counts_and_sum_low(departments_missing_in_location, 'department', 1, True)
departments_missing_in_location

,department,counts
0,Editorial,5.0
1,Marketing,5.0
2,Provisions,5.0
3,IT,4.0
4,Sales,4.0
5,Africa Program,2.0
6,"New York City or Paris, France",2.0
7,African Program,2.0
8,22 other departments below 1,22.0


In [14]:
departments_location = make_dataframe_based_on_counts_and_sum_low(postings, 'department', 50)
departments_location

,department,counts
0,Sales,551.0
1,Engineering,487.0
2,Marketing,401.0
3,Operations,270.0
4,IT,225.0
5,Development,146.0
6,Product,112.0
7,Information Technology,86.0
8,Design,76.0
9,Technology,76.0


In [15]:
# make plot with 2 subplots
plot_relation(departments_location, departments_missing_in_location,'department')


Department is missing randomly, there is no relation between location and department

In [16]:
missing_at_random.append('department')

#### Salary Range

In [17]:
missing_salary_range = make_df_missing_location(missing_location, 'salary_range')
missing_salary_range.shape

(21, 2)

In [18]:
missing = make_dataframe_based_on_counts_and_sum_low(missing_salary_range, 'salary_range', 1, True)
missing

,salary_range,counts
0,75000-80000,2.0
1,20 other salary_ranges below 1,20.0


In [19]:
all = make_dataframe_based_on_counts_and_sum_low(postings, 'salary_range', 30)
all

,salary_range,counts
0,0-0,142.0
1,40000-50000,66.0
2,30000-40000,55.0
3,25000-30000,37.0
4,45000-67000,37.0
5,30000-50000,32.0
6,868 other salary_ranges below 30,2499.0


In [20]:
plot_relation(all, missing, 'salary_range')

Salary Range is missing at random

In [21]:
missing_at_random.append('salary_range')

#### Telecommuting

In [22]:
telecommuting_missing_in_location = make_df_missing_location(missing_location, 'telecommuting')
telecommuting_missing_in_location

,telecommuting,counts
0,0,325
1,1,21


In [23]:
missing_tele = make_dataframe_based_on_counts_and_sum_low(telecommuting_missing_in_location, 'telecommuting', 1, True).iloc[0:2]
missing_tele

,telecommuting,counts
0,0,325.0
1,1,21.0


In [24]:
all_tele = make_dataframe_based_on_counts_and_sum_low(postings, 'telecommuting', 30).iloc[0:2]
all_tele

,telecommuting,counts
0,0,17113.0
1,1,767.0


In [25]:
fig = plot_relation(all_tele, missing_tele, 'telecommuting')

Telecommuting is NOT missing at random, as the correlation is similar (4% to 6%), it should be value '0' most of the time

In [26]:
not_missing_at_random.append('telecommuting')

#### Has Company Logo

In [27]:
has_company_logo_missing_in_location = make_df_missing_location(missing_location, 'has_company_logo')
has_company_logo_missing_in_location

,has_company_logo,counts
0,1,291
1,0,55


In [28]:
missing_company_logo = make_dataframe_based_on_counts_and_sum_low(has_company_logo_missing_in_location, 'has_company_logo', 1, True).iloc[0:2]
missing_company_logo

,has_company_logo,counts
0,1,291.0
1,0,55.0


In [29]:
all_company_logo = make_dataframe_based_on_counts_and_sum_low(postings, 'has_company_logo', 30).iloc[0:2]
all_company_logo

,has_company_logo,counts
0,1,14220.0
1,0,3660.0


In [30]:
plot_relation(all_company_logo, missing_company_logo, 'has_company_logo')

Has Company Logo is NOT missing at random (15% to 20.5%), it should be value '1' most of the time

In [31]:
not_missing_at_random.append('has_company_logo')

#### Has Questions

In [32]:
has_questions_missing_in_location = make_df_missing_location(missing_location, 'has_questions')
has_questions_missing_in_location

,has_questions,counts
0,0,174
1,1,172


In [33]:
missing_questions = make_dataframe_based_on_counts_and_sum_low(has_questions_missing_in_location, 'has_questions', 1, True).iloc[0:2]
missing_questions

,has_questions,counts
0,0,174.0
1,1,172.0


In [34]:
all_questions = make_dataframe_based_on_counts_and_sum_low(postings, 'has_questions', 30).iloc[0:2]
all_questions

,has_questions,counts
0,0,9088.0
1,1,8792.0


In [35]:
plot_relation(all_questions, missing_questions, 'has_questions')

Has Question is missing at random (we have a 50% chance to pick one out of two)

In [36]:
missing_at_random.append('has_questions')

#### Required Experience

In [37]:
required_experience_missing_in_location = make_df_missing_location(missing_location, 'required_experience')
required_experience_missing_in_location

,required_experience,counts
0,Mid-Senior level,33
1,Not Applicable,16
2,Associate,15
3,Internship,13
4,Entry level,12
5,Director,3
6,Executive,1


In [38]:
missing_required_experience = make_dataframe_based_on_counts_and_sum_low(required_experience_missing_in_location, 'required_experience', 0, True).iloc[0:7]
missing_required_experience

,required_experience,counts
0,Mid-Senior level,33.0
1,Not Applicable,16.0
2,Associate,15.0
3,Internship,13.0
4,Entry level,12.0
5,Director,3.0
6,Executive,1.0


In [39]:
all_required_experience = make_dataframe_based_on_counts_and_sum_low(postings, 'required_experience', 30).iloc[0:7]
all_required_experience

,required_experience,counts
0,Mid-Senior level,3809.0
1,Entry level,2697.0
2,Associate,2297.0
3,Not Applicable,1116.0
4,Director,389.0
5,Internship,381.0
6,Executive,141.0


In [40]:
plot_relation(all_required_experience, missing_required_experience, 'required_experience')

Required Experience is not missing at random, but we can't accurately pinpoint the probabilities, we know that 'Mid-Senior level' is missing for every third value

In [41]:
not_missing_at_random.append('required_experience')

#### Required Education

In [42]:
required_education_missing_in_location = make_df_missing_location(missing_location, 'required_education')
required_education_missing_in_location

,required_education,counts
0,Bachelor's Degree,33
1,High School or equivalent,13
2,Master's Degree,6
3,Unspecified,5
4,Associate Degree,2
5,Vocational,1
6,Some College Coursework Completed,1
7,Certification,1


In [43]:
missing_required_education = make_dataframe_based_on_counts_and_sum_low(required_education_missing_in_location, 'required_education', 0, True).iloc[0:8]
missing_required_education

,required_education,counts
0,Bachelor's Degree,33.0
1,High School or equivalent,13.0
2,Master's Degree,6.0
3,Unspecified,5.0
4,Associate Degree,2.0
5,Vocational,1.0
6,Some College Coursework Completed,1.0
7,Certification,1.0


In [44]:
all_required_education = make_dataframe_based_on_counts_and_sum_low(postings, 'required_education', 30).iloc[0:8]
all_required_education

,required_education,counts
0,Bachelor's Degree,5145.0
1,High School or equivalent,2080.0
2,Unspecified,1397.0
3,Master's Degree,416.0
4,Associate Degree,274.0
5,Certification,170.0
6,Some College Coursework Completed,102.0
7,Professional,74.0


In [45]:
plot_relation(all_required_education, missing_required_education, 'required_education')

Required Education is missing at Random, as there is almost no change from all data to the missing values

In [46]:
missing_at_random.append('required_education')

#### Industry

In [47]:
industry_missing_in_location = make_df_missing_location(missing_location, 'industry')
industry_missing_in_location

,industry,counts
0,Information Technology and Services,16
1,Internet,14
2,Computer Software,12
3,Marketing and Advertising,7
4,Design,5
5,Financial Services,5
6,Human Resources,4
7,Automotive,2
8,Hospital & Health Care,2
9,"Health, Wellness and Fitness",2


In [48]:
missing_industry = make_dataframe_based_on_counts_and_sum_low(industry_missing_in_location, 'industry', 2, True)
missing_industry

,industry,counts
0,Information Technology and Services,16.0
1,Internet,14.0
2,Computer Software,12.0
3,Marketing and Advertising,7.0
4,Design,5.0
5,Financial Services,5.0
6,Human Resources,4.0
7,27 other industrys below 2,35.0


In [49]:
all_industry = make_dataframe_based_on_counts_and_sum_low(postings, 'industry', 100)
all_industry

,industry,counts
0,Information Technology and Services,1734.0
1,Computer Software,1376.0
2,Internet,1062.0
3,Marketing and Advertising,828.0
4,Education Management,822.0
5,Financial Services,779.0
6,Hospital & Health Care,497.0
7,Consumer Services,358.0
8,Telecommunications,342.0
9,Oil & Energy,287.0


In [50]:
plot_relation(all_industry, missing_industry, 'industry')

Industry is missing at random, because 13*/16% is IT, 10/12% is Computer Science etc.

In [51]:
missing_at_random.append('industry')

#### Function

In [52]:
function_missing_in_location = make_df_missing_location(missing_location, 'function')
function_missing_in_location

,function,counts
0,Information Technology,14
1,Engineering,13
2,Administrative,10
3,Sales,10
4,Marketing,9
5,Design,4
6,General Business,3
7,Writing/Editing,3
8,Business Development,2
9,Other,2


In [53]:
missing_function = make_dataframe_based_on_counts_and_sum_low(function_missing_in_location, 'function', 2, True)
missing_function

,function,counts
0,Information Technology,14.0
1,Engineering,13.0
2,Administrative,10.0
3,Sales,10.0
4,Marketing,9.0
5,Design,4.0
6,General Business,3.0
7,Writing/Editing,3.0
8,13 other functions below 2,18.0


In [54]:
all_function = make_dataframe_based_on_counts_and_sum_low(postings, 'function', 300)
all_function

,function,counts
0,Information Technology,1749.0
1,Sales,1468.0
2,Engineering,1348.0
3,Customer Service,1229.0
4,Marketing,830.0
5,Administrative,630.0
6,Design,340.0
7,Health Care Provider,338.0
8,Other,325.0
9,Education,325.0


In [55]:
plot_relation(all_function, missing_function, 'function')

Function is missing at random

In [56]:
missing_at_random.append('function')

### Conclusion

In [57]:
missing_at_random, not_missing_at_random

(['department',
  'salary_range',
  'has_questions',
  'required_education',
  'industry',
  'function'],
 ['telecommuting', 'has_company_logo', 'required_experience'])

### Split Location

As we can see there is a lot missing at random and since we are only missing around 2% of all data we will drop these NaN values

In [58]:
# drop all rows with missing values in location
postings = postings.dropna(subset=['location'])
postings.shape

(17534, 18)

In [59]:
location_split = list(postings['location'].str.split(',').values)
location_split[40:45]

[['US', ' PA', ' Kutztown'],
 ['US', ' CA', ' Sacramento'],
 ['US'],
 ['US', ' MD', ' Columbia'],
 ['US', ' OK', ' Tulsa']]

we will strip the whitespaces at the end and beginning to trim the values

In [60]:
for i, location in enumerate(location_split):
    for j, loc in enumerate(location):
        location_split[i][j] = loc.strip()

Fill empty Fields (='') with the word 'Unspecified'

In [61]:
for i, location in enumerate(location_split):
    for j, element in enumerate(location):
        if element == '':
            location[j] = 'Unspecified'

In [62]:
location_split[:10]

[['US', 'NY', 'New York'],
 ['NZ', 'Unspecified', 'Auckland'],
 ['US', 'IA', 'Wever'],
 ['US', 'DC', 'Washington'],
 ['US', 'FL', 'Fort Worth'],
 ['US', 'MD', 'Unspecified'],
 ['DE', 'BE', 'Berlin'],
 ['US', 'CA', 'San Francisco'],
 ['US', 'FL', 'Pensacola'],
 ['US', 'AZ', 'Phoenix']]

#### Less then 3 Values

In [63]:
for i, location in enumerate(location_split):
    if len(location) < 3:
        print(f'{i}: {location}')

42: ['US']
172: ['US']
228: ['US']
363: ['US']
387: ['US']
492: ['US']
528: ['US']
851: ['AU']
880: ['US']
946: ['GR']
978: ['US']
1017: ['CA']
1342: ['HK']
1642: ['DE']
1678: ['GB']
1694: ['US']
1761: ['US']
1780: ['US']
1956: ['IN']
2153: ['US']
2216: ['US']
2238: ['US']
2256: ['US']
2296: ['US']
2381: ['US']
2525: ['US']
2615: ['US']
2721: ['US']
2889: ['US']
3134: ['GB']
3462: ['US']
3642: ['US']
3734: ['US']
3901: ['AU']
4070: ['US']
4086: ['US']
4131: ['GR']
4132: ['US']
4150: ['GB']
4331: ['GB']
4500: ['US']
4652: ['US']
5183: ['US']
5381: ['US']
5712: ['BE']
5845: ['CN']
7031: ['TH']
7276: ['US']
7286: ['US']
7312: ['QA']
7324: ['US']
7353: ['US']
7376: ['US']
7553: ['US']
7689: ['US']
7708: ['US']
7760: ['US']
7769: ['US']
7843: ['GR']
7901: ['NZ']
8599: ['GB']
8661: ['QA']
9740: ['GB']
9886: ['US']
10176: ['GB']
10217: ['US']
10303: ['US']
10315: ['US']
10650: ['GB']
10880: ['US']
11386: ['US']
11407: ['US']
11673: ['AU']
11777: ['IN']
11804: ['IL']
11851: ['US']
12766: ['US'

Check if there are datarows with 2 and with 1 value

In [64]:
# only countries
any(len(location) == 1 for location in location_split)

True

In [65]:
# countries and states
any(len(location) == 2 for location in location_split)

False

If there is less then 3 values given, then there is exactly 1 value given

In [66]:
# get all unique countries 
unique_countries = {}
for i, location in enumerate(location_split):
    if len(location) == 1:
        unique_countries[location[0]] = unique_countries.get(location[0], 0) + 1
unique_countries

{'US': 63,
 'AU': 3,
 'GR': 3,
 'CA': 2,
 'HK': 1,
 'DE': 2,
 'GB': 10,
 'IN': 2,
 'BE': 1,
 'CN': 1,
 'TH': 1,
 'QA': 2,
 'NZ': 1,
 'IL': 1,
 'BH': 1}

Because the missing values are missing at random, we cannot fill them with real data and so we use 'Unspecified'.

In [67]:
for i, location in enumerate(location_split):
    if len(location) == 1:
        location_split[i] += ['Unspecified']*2

Check if there are no datarows with less then 3 values

In [68]:
any(len(location) < 3 for location in location_split)

False

#### Exactly 3 Values

In [69]:
for i, location in enumerate(location_split):
    if len(location) == 3:
        print(f'{i}: {location}')

0: ['US', 'NY', 'New York']
1: ['NZ', 'Unspecified', 'Auckland']
2: ['US', 'IA', 'Wever']
3: ['US', 'DC', 'Washington']
4: ['US', 'FL', 'Fort Worth']
5: ['US', 'MD', 'Unspecified']
6: ['DE', 'BE', 'Berlin']
7: ['US', 'CA', 'San Francisco']
8: ['US', 'FL', 'Pensacola']
9: ['US', 'AZ', 'Phoenix']
10: ['US', 'NJ', 'Jersey City']
11: ['GB', 'LND', 'London']
12: ['US', 'CT', 'Stamford']
13: ['US', 'FL', 'Orlando']
14: ['AU', 'NSW', 'Sydney']
15: ['SG', '01', 'Singapore']
17: ['GB', 'SOS', 'Southend-on-Sea']
18: ['US', 'NY', 'New York']
19: ['US', 'PA', 'USA Northeast']
20: ['US', 'TX', 'Austin']
21: ['NZ', 'N', 'Auckland']
22: ['AE', 'Unspecified', 'Unspecified']
23: ['US', 'CA', 'Carlsbad']
24: ['GB', 'LND', 'London']
25: ['US', 'NY', 'New York']
26: ['SG', 'Unspecified', 'Unspecified']
27: ['AE', 'AZ', 'Abudhabi']
28: ['US', 'MO', 'St. Louis']
29: ['CA', 'ON', 'Toronto']
30: ['US', 'MA', 'Waltham']
31: ['US', 'KS', 'Unspecified']
32: ['US', 'WA', 'Everett']
33: ['US', 'CA', 'San Ramon']
3

That's good like that, we dont need to change anything

#### More then 3 Values

In [70]:
for i, location in enumerate(location_split):
    if len(location) > 3:
        print(f'{i}: {location}')

16: ['IL', 'Unspecified', 'Tel Aviv', 'Israel']
69: ['US', 'CA', 'Menlo Park', 'CA']
98: ['US', 'Unspecified', 'Stocton', 'CA']
213: ['US', 'CA', 'Bakersfield', 'CA / Mt. Poso']
217: ['SA', '01', 'Riyadh', 'Olaya']
311: ['EG', 'C', 'Cairo', 'Nasr City']
339: ['GR', 'I', 'Neo Iraklio', 'Athens']
433: ['GB', 'UKM', 'Stockholm', 'Sweden']
682: ['GB', 'Unspecified', 'Angel', 'London']
720: ['US', 'AL', 'Birmingham', 'Montgomery', 'Atlanta Ga']
964: ['US', 'CO', 'Boulder', 'CO']
1704: ['US', 'TX', 'Austin', 'San Antonio', 'Houston']
1989: ['EG', 'C', 'Cairo', 'Nasr City']
2067: ['US', 'NJ', 'Hillside', 'NJ']
2286: ['GR', 'I', 'Vrilissia', 'Greece']
2327: ['GR', 'E', 'Larisa', 'Volos']
2655: ['GR', 'I', 'Chalandri', 'Athens']
2706: ['US', 'NJ', 'Whitehouse station', 'Unspecified']
2906: ['US', 'DC', 'Washington', 'DC']
3018: ['US', 'CA', 'Salinas', 'CA']
3404: ['US', 'CA', 'Fresno', 'CA']
3582: ['SA', '01', 'Riyadh', 'Olaya']
4015: ['US', 'FL', 'Okeechobee', 'FL']
4057: ['GR', 'I', 'Athens',

We have some problems with this data:
- Duplicate Country and State
- 'Empty' (='Unspecified') values at the end of the row (as a 4th value e.g. 2706)
- Multiple Cities or States in 1 row

We will need 2 steps to solve this problem: 
1. Check for duplicates or 'Unspecified' in 4th column
2. Check for the long terms of states (e.g. CA = California, C = Cairo)
3. Make new row for each city or state

##### Duplicates and 'Unspecified'

In [71]:
for i, location in enumerate(location_split):
    # regex checks if state or country is in 4th row but only starting not in the middle of the string
    if len(location) > 3 and (re.search(f'^{location[1]}', location[3]) or re.search(f'^{location[0]}', location[3]) or location[-1] == 'Unspecified'):
        print(f'{i}: {location}')

69: ['US', 'CA', 'Menlo Park', 'CA']
213: ['US', 'CA', 'Bakersfield', 'CA / Mt. Poso']
964: ['US', 'CO', 'Boulder', 'CO']
2067: ['US', 'NJ', 'Hillside', 'NJ']
2706: ['US', 'NJ', 'Whitehouse station', 'Unspecified']
2906: ['US', 'DC', 'Washington', 'DC']
3018: ['US', 'CA', 'Salinas', 'CA']
3404: ['US', 'CA', 'Fresno', 'CA']
4015: ['US', 'FL', 'Okeechobee', 'FL']
4458: ['US', 'NJ', 'Lakewood', 'Brick', 'Toms River', 'Jackson', 'Unspecified']
4679: ['US', 'CA', 'Palo Alto', 'CA']
5252: ['US', 'CA', 'San Francisco', 'CA']
5736: ['US', 'CA', 'Bakersfield', 'CA']
6405: ['US', 'GA', 'Alpharetta', 'GA']
6645: ['US', 'CA', 'Palo Alto', 'Unspecified']
7743: ['US', 'CA', 'Fresno', 'CA']
9007: ['US', 'NY', 'Brooklyn', 'NY']
9072: ['US', 'CA', 'Bakersfield', 'CA / Mt. Poso']
9319: ['US', 'NY', 'Long Island City', 'NY']
11480: ['US', 'CA', 'Bakersfield', 'CA']
11495: ['US', 'CA', 'Bakersfield', 'CA']
11497: ['US', 'CA', 'Bakersfield', 'CA']
11549: ['US', 'MD', 'Baltimore', 'Unspecified']
12197: ['US

In [72]:
for i, location in enumerate(location_split):
    # regex checks if state or country is in 4th row but only starting not in the middle of the string
    if len(location) > 3 and (re.search(f'^{location[1]}', location[3]) or re.search(f'^{location[0]}', location[3]) or location[-1] == 'Unspecified'):
        location_split[i] = location[:3]

In [73]:
location_split[69]

['US', 'CA', 'Menlo Park']

##### Check for the long term of Country and States

In [74]:
location_split

[['US', 'NY', 'New York'],
 ['NZ', 'Unspecified', 'Auckland'],
 ['US', 'IA', 'Wever'],
 ['US', 'DC', 'Washington'],
 ['US', 'FL', 'Fort Worth'],
 ['US', 'MD', 'Unspecified'],
 ['DE', 'BE', 'Berlin'],
 ['US', 'CA', 'San Francisco'],
 ['US', 'FL', 'Pensacola'],
 ['US', 'AZ', 'Phoenix'],
 ['US', 'NJ', 'Jersey City'],
 ['GB', 'LND', 'London'],
 ['US', 'CT', 'Stamford'],
 ['US', 'FL', 'Orlando'],
 ['AU', 'NSW', 'Sydney'],
 ['SG', '01', 'Singapore'],
 ['IL', 'Unspecified', 'Tel Aviv', 'Israel'],
 ['GB', 'SOS', 'Southend-on-Sea'],
 ['US', 'NY', 'New York'],
 ['US', 'PA', 'USA Northeast'],
 ['US', 'TX', 'Austin'],
 ['NZ', 'N', 'Auckland'],
 ['AE', 'Unspecified', 'Unspecified'],
 ['US', 'CA', 'Carlsbad'],
 ['GB', 'LND', 'London'],
 ['US', 'NY', 'New York'],
 ['SG', 'Unspecified', 'Unspecified'],
 ['AE', 'AZ', 'Abudhabi'],
 ['US', 'MO', 'St. Louis'],
 ['CA', 'ON', 'Toronto'],
 ['US', 'MA', 'Waltham'],
 ['US', 'KS', 'Unspecified'],
 ['US', 'WA', 'Everett'],
 ['US', 'CA', 'San Ramon'],
 ['GB', 'LN

Writing all Countries that are in column 4 (location[3]) <br>
and also abbreviations of states (e.g. DC = D.C.)

In [75]:
countries = ['Israel', 'Sweden', 'Greece', 'Philippines', 'D.C.', 'California']

for i, location in enumerate(location_split):
    if len(location) > 3 and location[3] in countries:
        print(f'{i}: {location}')

16: ['IL', 'Unspecified', 'Tel Aviv', 'Israel']
433: ['GB', 'UKM', 'Stockholm', 'Sweden']
2286: ['GR', 'I', 'Vrilissia', 'Greece']
6733: ['PH', 'Unspecified', 'Makati', 'Philippines']
8810: ['US', 'DC', 'Washington', 'D.C.']
9619: ['GB', 'UKM', 'Stockholm', 'Sweden']
10676: ['GB', 'Unspecified', 'Stockholm', 'Sweden']


In [76]:
for i, location in enumerate(location_split):
    if len(location) > 3 and location[3] in countries:
        location_split[i] = location[:3]

While checking i found out, that there are some states in column 4 and column 2 is 'unspecified'

In [77]:
for i, location in enumerate(location_split):
    if len(location) > 3 and (location[1] == 'Unspecified' and location[3].isupper() and len(location[3]) == 2):
        print(f'{i}: {location}')

98: ['US', 'Unspecified', 'Stocton', 'CA']
4160: ['CA', 'Unspecified', 'Rio Rancho', 'NM']
9668: ['US', 'Unspecified', 'Stocton', 'CA']
11251: ['US', 'Unspecified', 'Denver', 'CO']
11792: ['US', 'Unspecified', 'Portsmouth', 'NH']
12555: ['US', 'Unspecified', 'Brooklyn', 'NY']


In [78]:
for i, location in enumerate(location_split):
    if len(location) > 3 and (location[1] == 'Unspecified' and location[3].isupper() and len(location[3]) == 2):
        location_split[i] = location[0] + location[3] + location[2]

While checking I found some string values, which seem to consist of country + state + city, i will transform these into a list

In [79]:
for i, location in enumerate(location_split):
    if type(location) == str:
        print(f'{i}: {location}')

98: USCAStocton
4160: CANMRio Rancho
9668: USCAStocton
11251: USCODenver
11792: USNHPortsmouth
12555: USNYBrooklyn


In [80]:
for i, location in enumerate(location_split):
    if type(location) == str:
        location_split[i] = [location[:2], location[2:4], location[4:]]

In [81]:
count = 0
for i, location in enumerate(location_split):
    if len(location) > 3:
        print(f'{i}: {location}')
        count += 1
        
count

217: ['SA', '01', 'Riyadh', 'Olaya']
311: ['EG', 'C', 'Cairo', 'Nasr City']
339: ['GR', 'I', 'Neo Iraklio', 'Athens']
682: ['GB', 'Unspecified', 'Angel', 'London']
720: ['US', 'AL', 'Birmingham', 'Montgomery', 'Atlanta Ga']
1704: ['US', 'TX', 'Austin', 'San Antonio', 'Houston']
1989: ['EG', 'C', 'Cairo', 'Nasr City']
2327: ['GR', 'E', 'Larisa', 'Volos']
2655: ['GR', 'I', 'Chalandri', 'Athens']
3582: ['SA', '01', 'Riyadh', 'Olaya']
4057: ['GR', 'I', 'Athens', 'Rouf']
4265: ['GR', 'I', 'Kifisia', 'Athens']
4328: ['US', 'MI', 'Hartford', 'Paw Paw', 'Lawton']
4411: ['US', 'CA', 'San Jose', 'SFO', 'Austin', 'OH']
4516: ['GR', 'I', 'Kifisia', 'Athens']
4519: ['GR', 'I', 'Kifisia', 'Athens']
4569: ['GB', 'Unspecified', 'Grange Moore', 'Wakefield']
4580: ['US', 'FL', 'Central Fl', 'Nort Fl', 'Georgia', 'Alabama', 'Tennessee', 'South Carolina', 'North Carolina']
5251: ['GR', 'I', 'Acharnes', 'Athens']
6196: ['GR', 'I', 'Vrilissia', 'Athens']
6903: ['GR', 'I', 'Vrilissia', 'Athens']
8008: ['PH',

74

Looking at the 74 Datarows:
- 8850: seems false, Great Britain and Madrid (is in Spain) in the same row makes no sense
- 17233: 'Asbury Park NJ', 'Belmar NJ', 'Toms River NJ.' are in New Jersey and can be deleted from this value
- 13541: 'partially in the UK' is wrong, munich is in germany

In [82]:
location_split[17233] = location_split[17233][:-3]
location_split[13541] = location_split[13541][:-1]

combine the new dataset and apply following rules:
- make new row for each city too much
- delete row 8850